<a href="https://colab.research.google.com/github/myrah/AAI2025/blob/dev/AdAgent/Hands_on_Ad_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building an Ad Optimization Agent is an excellent, practical use case for AI agents\! Since the core functionality will involve complex, multi-step decision-making, the **LangGraph** framework (an extension of LangChain) is ideal for defining the stateful, cyclical workflow.

Here is a simplified Python workshop code template using **LangGraph** and **LangChain** components. This example focuses on the agent's decision-making flow rather than connecting to live ad platform APIs, which would be proprietary and outside a workshop's scope.

You'll need to install the necessary libraries and set your API key (e.g., for an OpenAI or Anthropic LLM).

```bash
pip install langgraph langchain langchain_openai pydantic
```

## Python Workshop Code: Ad Optimization Agent

The agent will follow a core decision loop: **Gather Data** $\rightarrow$ **Analyze** $\rightarrow$ **Decide Action** $\rightarrow$ **Execute Tool** $\rightarrow$ **Repeat/Finish**.

### 1\. Imports and State Definition

We define the `AgentState` to hold the data and decision history throughout the graph's execution.

In [1]:
pip install langgraph langchain langchain_openai pydantic

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 7.4 MB/s eta 0:00:00


In [2]:
import os
import operator
from typing import TypedDict, Annotated, List

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

from google.colab import userdata

# --- Environment Setup (Replace with your actual key) ---
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_EDU_API_KEY")

# 1. Define the Agent State
# This represents the information passed between nodes in the graph
class AgentState(TypedDict):
    """
    Represents the state of our ad optimization agent.
    - messages: A list of messages/history.
    - campaign_data: The current (simulated) ad campaign metrics.
    - next_action: The recommended action from the analysis.
    """
    messages: Annotated[List[BaseMessage], operator.add]
    campaign_data: dict
    next_action: str

# 2. Initialize LLM
# We'll use a powerful model for the reasoning engine
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

-----

### 2\. Define Custom Tools

We create "tools" (Python functions) that the LLM can decide to use to interact with the external world (e.g., ad platform APIs). In a workshop, these are simulated.

In [3]:
# --- Simulated Tools for Ad Optimization ---

@tool
def adjust_bid(campaign_id: str, new_bid_amount: float) -> str:
    """Adjusts the bid for a specific ad campaign to the new amount."""
    if new_bid_amount > 5.00:
        return f"Bid for Campaign '{campaign_id}' adjusted to ${new_bid_amount:.2f}. (Simulated: High bid warning!)"
    return f"Bid for Campaign '{campaign_id}' adjusted to ${new_bid_amount:.2f} successfully."

@tool
def pause_ad_group(ad_group_id: str) -> str:
    """Pauses an underperforming ad group by its ID."""
    return f"Ad Group '{ad_group_id}' paused successfully due to poor performance."

@tool
def request_new_creatives(campaign_id: str) -> str:
    """Requests new ad creative assets from the creative team."""
    return f"New creative request submitted for Campaign '{campaign_id}'. Awaiting design feedback."

ad_optimization_tools = [adjust_bid, pause_ad_group, request_new_creatives]

-----

### 3\. Define Graph Nodes (Steps)

The nodes are the functions that run at each step of the agent's workflow.

In [9]:
def fetch_campaign_data(state: AgentState) -> dict:
    """Simulates fetching the latest campaign data."""
    print("--- FETCHING DATA ---")
    # In a real scenario, this would call an Ad API (e.g., Google Ads, Meta)
    # This is a fixed, simulated dataset for the workshop
    simulated_data = {
        "Campaign-2024-Q4": {
            "Budget": 1000, "Spend": 850, "Impressions": 50000,
            "Clicks": 500, "Conversions": 5, "CPA": 170.00, "Target_CPA": 50.00,
            "Ad_Groups": {
                "AG-101": {"Status": "Active", "CPA": 25.00, "Bid": 2.50},
                "AG-102": {"Status": "Active", "CPA": 450.00, "Bid": 3.00} # Poor performer
            }
        }
    }

    analysis_prompt = (
        "Ad Campaign Data for Optimization:\n"
        f"{simulated_data}\n\n"
        "**Optimization Goal:** Reduce overall CPA (Cost Per Acquisition) to be closer to the Target CPA ($50.00) "
        "and maximize conversions. Analyze the data and recommend the *single best action* "
        "using one of the provided tools (adjust_bid, pause_ad_group, request_new_creatives). "
        "Your final output should be ONLY the recommended action as a message for the user, "
        "or a clear instruction for the next internal step."
    )

    # Prepend the analysis prompt to the messages for the LLM
    new_messages = [HumanMessage(content=analysis_prompt)]

    return {"messages": new_messages, "campaign_data": simulated_data}

def agent_reasoning(state: AgentState) -> dict:
    """The LLM reasons and decides the next step (tool call or final answer)."""
    print("--- AGENT REASONING ---")

    # Bind the tools to the LLM
    llm_with_tools = llm.bind_tools(ad_optimization_tools)

    # Define a system prompt to guide the LLM's role
    system_prompt = (
        "You are an expert Ad Campaign Optimization Agent. "
        "Your goal is to analyze the provided campaign data and decide the optimal next action. "
        "You must use a tool if an optimization is possible. "
        "If no tool is needed or you have executed a tool, provide a final, concise update."
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("placeholder", "{messages}")
    ])

    # Run the LLM to get a decision
    chain = prompt | llm_with_tools
    response = chain.invoke(state)

    # Determine if a tool call was made
    if response.tool_calls:
        # If a tool is called, the next step is to execute it.
        return {"messages": [response], "next_action": "call_tool"}
    else:
        # If no tool is called, the reasoning is the final response.
        return {"messages": [response], "next_action": "FINISH"}


def execute_tools(state: AgentState) -> dict:
    """Executes the tool call(s) decided by the agent_reasoning step."""
    print("--- EXECUTING TOOL ---")

    tool_calls = state["messages"][-1].tool_calls
    tool_results = []

    # Find and execute the function corresponding to the tool call
    for call in tool_calls:
        tool_name = call["name"]
        tool_args = call["args"]
        tool_call_id = call["id"] # Get the tool call ID

        # Simple lookup and execution (in a real app, use the ToolExecutor)
        tool_to_run = next(
            (t for t in ad_optimization_tools if t.name == tool_name), None
        )

        if tool_to_run:
            try:
                result = tool_to_run.invoke(tool_args)
                # Format the tool result as a ToolMessage
                tool_results.append({
                    "type": "tool_output",
                    "tool_call_id": tool_call_id, # Include the tool call ID
                    "content": result
                })
            except Exception as e:
                 # Handle tool execution errors and return an error message
                 tool_results.append({
                    "type": "tool_output",
                    "tool_call_id": tool_call_id, # Include the tool call ID
                    "content": f"Error executing tool {tool_name}: {e}"
                })


    # Add tool results to the state for the LLM to process next
    # Need to format the tool results into a list of BaseMessage
    tool_messages = [HumanMessage(content=str(r), name="tool_execution") for r in tool_results]

    return {"messages": tool_messages}

def decide_next_step(state: AgentState) -> str:
    """Conditional edge: decides whether to continue analysis or finish."""
    print(f"--- DECIDING NEXT STEP ---")
    print(f"state['next_action']: {state.get('next_action')}") # Use .get() for safety

    next_action = state.get("next_action")

    if next_action == "call_tool":
        print("Returning 'call_tool'") # Corrected return value
        return "call_tool"
    elif next_action == "FINISH":
        print("Returning 'end'")
        return "end"
    else:
        # After tool execution, go back to reasoning to summarize the result
        print("Returning 'agent_reasoning'")
        return "agent_reasoning"

In [10]:
from langchain_core.messages import ToolMessage

def fetch_campaign_data(state: AgentState) -> dict:
    """Simulates fetching the latest campaign data."""
    print("--- FETCHING DATA ---")
    # In a real scenario, this would call an Ad API (e.g., Google Ads, Meta)
    # This is a fixed, simulated dataset for the workshop
    simulated_data = {
        "Campaign-2024-Q4": {
            "Budget": 1000, "Spend": 850, "Impressions": 50000,
            "Clicks": 500, "Conversions": 5, "CPA": 170.00, "Target_CPA": 50.00,
            "Ad_Groups": {
                "AG-101": {"Status": "Active", "CPA": 25.00, "Bid": 2.50},
                "AG-102": {"Status": "Active", "CPA": 450.00, "Bid": 3.00} # Poor performer
            }
        }
    }

    analysis_prompt = (
        "Ad Campaign Data for Optimization:\n"
        f"{simulated_data}\n\n"
        "**Optimization Goal:** Reduce overall CPA (Cost Per Acquisition) to be closer to the Target CPA ($50.00) "
        "and maximize conversions. Analyze the data and recommend the *single best action* "
        "using one of the provided tools (adjust_bid, pause_ad_group, request_new_creatives). "
        "Your final output should be ONLY the recommended action as a message for the user, "
        "or a clear instruction for the next internal step."
    )

    # Prepend the analysis prompt to the messages for the LLM
    new_messages = [HumanMessage(content=analysis_prompt)]

    return {"messages": new_messages, "campaign_data": simulated_data}

def agent_reasoning(state: AgentState) -> dict:
    """The LLM reasons and decides the next step (tool call or final answer)."""
    print("--- AGENT REASONING ---")

    # Bind the tools to the LLM
    llm_with_tools = llm.bind_tools(ad_optimization_tools)

    # Define a system prompt to guide the LLM's role
    system_prompt = (
        "You are an expert Ad Campaign Optimization Agent. "
        "Your goal is to analyze the provided campaign data and decide the optimal next action. "
        "You must use a tool if an optimization is possible. "
        "If no tool is needed or you have executed a tool, provide a final, concise update."
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("placeholder", "{messages}")
    ])

    # Run the LLM to get a decision
    chain = prompt | llm_with_tools
    response = chain.invoke(state)

    # Determine if a tool call was made
    if response.tool_calls:
        # If a tool is called, the next step is to execute it.
        return {"messages": [response], "next_action": "call_tool"}
    else:
        # If no tool is called, the reasoning is the final response.
        return {"messages": [response], "next_action": "FINISH"}


def execute_tools(state: AgentState) -> dict:
    """Executes the tool call(s) decided by the agent_reasoning step."""
    print("--- EXECUTING TOOL ---")

    tool_calls = state["messages"][-1].tool_calls
    tool_results = []

    # Find and execute the function corresponding to the tool call
    for call in tool_calls:
        tool_name = call["name"]
        tool_args = call["args"]
        tool_call_id = call["id"] # Get the tool call ID

        # Simple lookup and execution (in a real app, use the ToolExecutor)
        tool_to_run = next(
            (t for t in ad_optimization_tools if t.name == tool_name), None
        )

        if tool_to_run:
            try:
                result = tool_to_run.invoke(tool_args)
                # Format the tool result as a ToolMessage
                tool_results.append(ToolMessage(
                    content=result,
                    tool_call_id=tool_call_id # Include the tool call ID
                ))
            except Exception as e:
                 # Handle tool execution errors and return an error message
                 tool_results.append(ToolMessage(
                    content=f"Error executing tool {tool_name}: {e}",
                    tool_call_id=tool_call_id # Include the tool call ID
                ))


    # Add tool results to the state for the LLM to process next
    return {"messages": tool_results}

def decide_next_step(state: AgentState) -> str:
    """Conditional edge: decides whether to continue analysis or finish."""
    print(f"--- DECIDING NEXT STEP ---")
    print(f"state['next_action']: {state.get('next_action')}") # Use .get() for safety

    next_action = state.get("next_action")

    if next_action == "call_tool":
        print("Returning 'call_tool'") # Corrected return value
        return "call_tool"
    elif next_action == "FINISH":
        print("Returning 'end'")
        return "end"
    else:
        # After tool execution, go back to reasoning to summarize the result
        print("Returning 'agent_reasoning'")
        return "agent_reasoning"

-----

### 4\. Build and Run the Graph

We define the flow using LangGraph's `StateGraph`.

In [11]:
# 5. Build the LangGraph Workflow
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("fetch_data", fetch_campaign_data)
workflow.add_node("agent_reasoning", agent_reasoning)
workflow.add_node("execute_tools", execute_tools)

# Set the start point
workflow.set_entry_point("fetch_data")

# Add edges
# From fetch_data, always go to reasoning
workflow.add_edge("fetch_data", "agent_reasoning")

# Conditional edge from reasoning to decide if it's a tool call or the end
workflow.add_conditional_edges(
    "agent_reasoning",
    decide_next_step,
    {"call_tool": "execute_tools", "end": END}
)

# After executing a tool, cycle back to reasoning to formulate a final summary
workflow.add_edge("execute_tools", "agent_reasoning")

# Compile the graph
app = workflow.compile()

# 6. Run the Agent
print("--- STARTING AD OPTIMIZATION AGENT RUN ---")

# The agent runs autonomously until it hits the END node
# It starts by fetching data, which primes the first message.
final_state = app.invoke(
    {"messages": [], "campaign_data": {}, "next_action": "start"},
    config={"recursion_limit": 50}
)

# 7. Print Final Result
final_message = final_state["messages"][-1].content
print("\n" + "="*50)
print("AGENT FINAL RECOMMENDATION & SUMMARY:")
print(final_message)
print("="*50)

# Optional: Visualize the graph (requires pydot/graphviz)
# from IPython.display import Image
# Image(app.get_graph().draw_png())

--- STARTING AD OPTIMIZATION AGENT RUN ---
--- FETCHING DATA ---
--- AGENT REASONING ---
--- DECIDING NEXT STEP ---
state['next_action']: call_tool
Returning 'call_tool'
--- EXECUTING TOOL ---
--- AGENT REASONING ---
--- DECIDING NEXT STEP ---
state['next_action']: FINISH
Returning 'end'

AGENT FINAL RECOMMENDATION & SUMMARY:
Ad Group 'AG-102' has been paused due to poor performance.


-----

The video provided below demonstrates building a more advanced research agent using LangChain and LangGraph, offering a great visual example of the framework's capabilities.

[Python Advanced AI Agent Tutorial - LangGraph, LangChain, Firecrawl & More\!](https://www.youtube.com/watch?v=xekw62yQu14&pp=0gcJCf8Ao7VqN5tD) This video is relevant as it provides a practical tutorial on building a complex, multi-step AI agent using the LangGraph and LangChain frameworks, which are central to the workshop code.